# RAG Pipeline for Smart Contract Generation

This notebook demonstrates the complete workflow of converting legal documents into smart contracts using RAG (Retrieval Augmented Generation) and LLM.

In [12]:
from rag_pipeline import RAGPipeline
from pathlib import Path
from ctransformers import AutoModelForCausalLM

# Initialize RAG Pipeline
rag = RAGPipeline()

# Load contract templates
templates_dir = Path('../templates')
rag.load_contract_templates(str(templates_dir))

# Initialize LLM
llm = AutoModelForCausalLM.from_pretrained(
    '../models/codellama-7b.Q5_K_M.gguf',
    model_type='llama',
    gpu_layers=50,
    context_length=4096
)


In [13]:
# Load and process rental agreement
def read_rental_agreement(file_path, n=0):
    with open(file_path, 'r') as file:
        agreements = file.read().split('---')
        return agreements[n].strip()

rental_agreement = read_rental_agreement('../docs/rentals.txt')
processed_sections = rag.process_document('../docs/rentals.txt')
print(f'Processed {len(processed_sections)} sections from the rental agreement')


Processed 1 sections from the rental agreement


In [14]:
# Load contract generation prompt
with open('../docs/contract_prompt.txt', 'r') as file:
    contract_prompt = file.read()

# Prepare the combined prompt
prompt = f"""
{contract_prompt}

Here is a sample rental agreement for reference:
{rental_agreement}

Solidity contract:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

// Begin implementation below:
"""

# Configure generation parameters
generation_config = {
    'temperature': 0.2,
    'top_p': 0.95,
    'max_new_tokens': 2500,
    'stop': ['// End of contract', '```', '// End implementation'],
    'repetition_penalty': 1.1
}


In [15]:
# Generate initial contract from LLM
initial_contract = llm(prompt, **generation_config)
print('Generated initial contract:')
print(initial_contract)


Generated initial contract:
contract RentalAgreement {
    address public lessor;
    address public lessee;
    string public propertyAddress;
    uint256 public rentAmount;
    uint256 public paymentFrequency;
    uint256 public agreementEndDate;
    mapping(address => bool) payments;
    
    event RentPaid(address indexed _sender, address indexed _receiver, uint256 _amount);
    event AgreementTerminated(address indexed _sender, address indexed _receiver, string _reason);
    
    constructor(address _lessor, address _lessee, string memory _propertyAddress, uint256 _rentAmount, uint256 _paymentFrequency, uint256 _agreementEndDate) {
        lessor = _lessor;
        lessee = _lessee;
        propertyAddress = _propertyAddress;
        rentAmount = _rentAmount;
        paymentFrequency = _paymentFrequency;
        agreementEndDate = _agreementEndDate;
    }
    
    modifier onlyLessor() {
        require(msg.sender == lessor, "Only the lessor can call this function.");
        _;
 

In [16]:
# Assemble final contract using RAG
final_contract = rag.assemble_contract(processed_sections)
print('Assembled final contract with RAG:')
print(final_contract)


Assembled final contract with RAG:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract RentalAgreement {
    address public lessor;
    address public lessee;
    string public propertyAddress;
    uint256 public rentAmount;
    uint256 public paymentFrequency;
    uint256 public agreementEndDate;
    mapping(uint => bool) payments;

    event RentPaid(address indexed sender, uint256 amount);
    event AgreementTerminated(address indexed sender);
    event PaymentVerified(address indexed sender, uint256 paymentNumber);

    constructor(
        address _lessor,
        address _lessee,
        string memory _propertyAddress,
        uint256 _rentAmount,
        uint256 _paymentFrequency,
        uint256 _agreementEndDate
    ) {
        lessor = _lessor;
        lessee = _lessee;
        propertyAddress = _propertyAddress;
        rentAmount = _rentAmount;
        paymentFrequency = _paymentFrequency;
        agreementEndDate = _agreementEndDate;
    }

    function payRe